In [ ]:
import wandb

# This is importent when we want to call this as a python script, because jupyter naturally has a higher recursion depth
from icecream import ic
import os
import sys
sys.setrecursionlimit(3000)

# Print the PID when using nohup
ic(os.getpid())

In [ ]:
from gymportal.data.ev_generators import get_standard_generator, RealWorldGenerator
from acnportal.acnsim import Linear2StageBattery
from gymportal.data.battery_generators import CustomizableBatteryGenerator
from gymportal.sim import get_charging_network, Recomputer, EvaluationSimulator, SimGenerator
from datetime import datetime, timedelta

import pytz
timezone = pytz.timezone("America/Los_Angeles")


charging_network = get_charging_network('simple_acn', basic_evse=True, voltage=208,
                                        network_kwargs={
                                            'station_ids': ['CA-504', 'CA-503', 'CA-502', 'CA-501'],
                                            # 'station_ids': ['CA-501'],
                                            "aggregate_cap": 32 * 208 / 1000})

# charging_network = get_charging_network('caltech', basic_evse=True, voltage=208,
#                                         network_kwargs={"transformer_cap": 150})

battery_generator = CustomizableBatteryGenerator(
    voltage=208,
    period=1,
    battery_types=[
        Linear2StageBattery],
    max_power_function="normal",
)

# ev_generator = RealWorldGenerator(battery_generator=battery_generator, site='caltech', period=1)
ev_generator = get_standard_generator(
    'caltech', battery_generator, seed=42, frequency_multiplicator=10, duration_multiplicator=2)

# TODO Use time intervals and GMMs from https://github.com/chrisyeh96/sustaingym/blob/main/sustaingym/envs/evcharging/utils.py
# I.e., train on generated data, evaluate on new generated data and real data from the same interval
# optional: compare to "out-of-distribution" data from different interval

train_generator = SimGenerator(
    charging_network=charging_network,
    simulation_days=7,
    n_intervals=10,
    start_date=timezone.localize(datetime(2019, 1, 1)),
    ev_generator=ev_generator,
    recomputer=Recomputer(recompute_interval=10, sparse=True),
    sim_class=EvaluationSimulator,
)

ic(train_generator.end_date + timedelta(days=1))

eval_generator = SimGenerator(
    charging_network=charging_network,
    simulation_days=7 * 4,
    n_intervals=1,
    start_date=train_generator.end_date + timedelta(days=1),
    ev_generator=ev_generator,
    recomputer=Recomputer(recompute_interval=10, sparse=True),
    sim_class=EvaluationSimulator,
)

ic(eval_generator.end_date + timedelta(days=1))

validation_generator = SimGenerator(
    charging_network=charging_network,
    simulation_days=14,
    n_intervals=1,
    start_date=eval_generator.end_date + timedelta(days=1),
    ev_generator=ev_generator,
    recomputer=Recomputer(recompute_interval=10, sparse=True),
    sim_class=EvaluationSimulator,
)

ic(validation_generator.end_date + timedelta(days=1))
pass

In [ ]:
from src.pv.pv import read_pv_data

df_pv = read_pv_data("../pv_150kW.csv")
df_pv.describe()

In [4]:
from gymportal.environment import *
from src.pv.observations import pv_observation_mean
from src.pv.rewards import *
from src.pv.metrics import *
from gymportal.evaluation import *


observation_objects = [
    charging_rates_observation_normalized(),
    percentage_of_magnitude_observation(),
    diff_pilots_charging_rates_observation_normalized(),
    cyclical_minute_observation(),
    cyclical_day_observation(),
    cyclical_month_observation(),
    cyclical_minute_observation_stay(), # TODO Use the minutes directly
    energy_delivered_observation_normalized(),
    num_active_stations_observation_normalized(),
    pilot_signals_observation_normalized(),
    pv_observation_mean(df_pv),
]

reward_objects = [
    pv_utilization_reward(df_pv),
    # grid_use_penalty(df_pv),
    unused_pv_penalty(df_pv),
    charging_reward(),
    # soft_charging_reward_pv_weighted(df_pv, transformer_cap=150),
]

metrics = {
    "SoC >= 90%": percentage_soc,
    "mean SoC": mean_soc,
    "median SoC": median_soc,
    "prop feasible steps": proportion_of_feasible_charging,
    "prop feasible charge": proportion_of_feasible_charge,
    "pv utilization": lambda sim: pv_utilization_metric(sim, df_pv),
    "grid usage": lambda sim: grid_use_metric(sim, df_pv),
    "unused pv": lambda sim: unused_pv_metric(sim, df_pv),
}

In [5]:
# import dill as pickle

# with open("../caltech_#stations=54_#days=7_#intervals=46_seed=8734956.pkl", "rb") as file:
#     train_generator = pickle.load(file)

In [ ]:
train_generator.seed = 8734956
_ = train_generator.reset()

iter = 0

while train_generator._current_date != train_generator.start_date:
    _ = train_generator.next()

    ic(iter)
    ic(train_generator._current_date)
    iter += 1

In [ ]:
steps_per_epoch = 0
for eval_sim in train_generator._sim_memory:
    steps_per_epoch += len(eval_sim.event_queue.queue)

ic(steps_per_epoch)

In [ ]:
eval_generator.seed = 8734956
_ = eval_generator.reset()

iter = 0

while eval_generator._current_date != eval_generator.start_date:
    _ = eval_generator.next()

    ic(iter)
    ic(eval_generator._current_date)
    iter += 1
    
steps_per_epoch_eval = 0
for eval_sim in eval_generator._sim_memory:
    steps_per_epoch_eval += len(eval_sim.event_queue.queue)

ic(steps_per_epoch_eval)

In [8]:
train_config = {"observation_objects": observation_objects, "action_object": zero_centered_single_charging_schedule_normalized(),
                "reward_objects": reward_objects,
                "simgenerator": train_generator,
                "meet_constraints": True}

eval_config = train_config | {'simgenerator': eval_generator}
validation_config = train_config | {'simgenerator': validation_generator}

In [11]:
from acnportal.algorithms import UncontrolledCharging, SortedSchedulingAlgo, last_come_first_served, \
    first_come_first_served

model = ACNSchedule(SortedSchedulingAlgo(first_come_first_served))

In [ ]:
from tqdm import tqdm
import pandas as pd

def create_df(model, name: str, env, steps_per_epoch):
    df_list = []
    
    done = False
    old_obs, _ = env.reset()

    for _ in tqdm(range(steps_per_epoch)):

        iface = env.unwrapped.interface
        action = model.get_action(old_obs, iface)

        new_obs, rew, terminated, truncated, _ = env.step(
            action)
        done = terminated or truncated

        df_list.append([old_obs.tolist(), action.tolist(), rew, done])

        if done:
            new_obs, _ = env.reset()
            done = False
            
        old_obs = new_obs
        
    df = pd.DataFrame(df_list, columns=['observation', 'action', 'reward', 'done'])
    df.to_parquet(f'{name}.parquet.gzip', compression='gzip')
    

In [ ]:
from src.cleanRL.environment import make_env

create_df(model, "FCFS_gen-_training", make_env(train_config, 0.99, 0)(), steps_per_epoch)
create_df(model, "FCFS_gen-_validation", make_env(eval_config, 0.99, 0)(), steps_per_epoch_eval)